## 2.8 Run-times

We determine the complexity of an algorithm from its English description.
Later, after implementing it in Python, we can measure the run-times
for ever-growing inputs to check the actual growth rate against what the
complexity analysis predicted.

The run-time depends on the hardware, operating system and Python interpreter
that execute the code, so you'll get different timings from mine
if you run the code cells in this notebook.
The run-times also depend on what other processes the computer is executing,
so they change every time we run the code.

We can measure the run-time of some code with the IPython command `%timeit`.
Instructions starting with a percentage sign aren't part of the Python language:
they're direct commands to the IPython interpreter.

The next cell will take some seconds to run (I'll explain why further below).
While code is running, the notebook has an asterisk instead of a number
in the brackets to the left of the code cell.
Once the code finishes executing, the asterisk becomes a number.

In [1]:
%timeit 2 + 7

3.63 ns ± 0.0203 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


Adding 2 and 7 takes a few nanoseconds (abbreviated ns).
The run-time varies each time the code cell is run, depending on the CPU load,
so as I write this text I don't know in advance the run-time you will see.
In the following I assume computing `2 + 7` takes 8 nanoseconds.

Other abbreviations you may see in the run-time report are
's' (seconds), 'ms' (milliseconds) and 'µs' (microseconds).
One second is a thousand milliseconds;
one millisecond is a thousand microseconds;
one microsecond is a thousand nanoseconds.
In other words, 1&nbsp;ms = $10^{−3}$&nbsp;s, 1&nbsp;µs = $10^{−6}$&nbsp;s and 1&nbsp;ns = $10^{−9}$&nbsp;s.

Measuring a very short lapse of time is prone to
significant measurement errors, so `%timeit` executes the code multiple times,
measures the total run-time, and divides it by the number of iterations
to get a more precise value. On my machine, the addition was computed 100 million times,
but on yours it may have been fewer or more times:
the IPython interpreter automatically chooses the number of loops.
Finally, to reduce the effect of other processes running at the same time,
`%timeit` runs everything seven times and takes the average.

Even though addition is very fast, running the code cell takes several seconds.
Assuming addition takes 8&nbsp;ns, the cell executes in
7 runs × 100,000,000 loops × 8&nbsp;ns = 7 × 100 × 8&nbsp;ms = 5,600&nbsp;ms = 5.6&nbsp;s.
Fortunately, we can reduce the time waiting for a result by
setting the number of runs and loops with options `-r` and `-n`, respectively.

In [2]:
%timeit -r 5 -n 1000000 2 + 7

3.66 ns ± 0.0334 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)


I reduce the number of runs to five and the number of loops to one million.
Now the cell runs in milliseconds: 5 × 1,000,000 × 8&nbsp;ns = 5 × 1 × 8&nbsp;ms = 40&nbsp;ms.

The time measured for addition may differ from previously,
e.g. it may now be 7.5&nbsp;ns or 8.5&nbsp;ns instead of 8&nbsp;ns,
because reducing the number of runs and loops reduces accuracy.
That's OK, as I'll explain in a minute.

Modern processors add two 64-bit numbers in hardware,
so the very short time is not a surprise.
Let's try some inputs that don't fit in 64&nbsp;bits.

In [3]:
%timeit -r 5 -n 1000000 (2 ** 64 + 1) + (2 ** 64 + 2)

3.65 ns ± 0.0309 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)


This took about the same time! That can't be right. Let's try a different way.

In [4]:
left = 2
right = 7
%timeit -r 5 -n 1000000 left + right
left = 2**64 + 1
right = 2**64 + 2
%timeit -r 5 -n 1000000 left + right

12.6 ns ± 0.357 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
24.3 ns ± 0.498 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)


The times look right now: adding longer numbers takes more time, as we'd expect.
Before, the interpreter figured out I was adding two constant values
and pre-computed the sum, as it wouldn't change.
I was just measuring the time to retrieve a value from memory,
which is always the same, no matter how small or large the value is.
The interpreter can't make such optimisations when adding
variables because their values may change.

<div class="alert alert-warning">
<strong>Note:</strong> When measuring the run-time of an expression, use variables instead of literals.
</div>

As I mentioned in the previous section,
'constant time' doesn't mean that *all* operations take the same time,
but rather that *each* operation takes the same time for small and large inputs.
For example, I expect that dividing two numbers takes longer than adding them.

In [5]:
left = 9876543210
right = 123456789
%timeit -r 5 -n 1000000 left + right
%timeit -r 5 -n 1000000 left // right

23.4 ns ± 0.232 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
29.1 ns ± 0.185 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)


We can also measure the run-time of functions we define.

In [6]:
def brick_volume(length: int, width: int, height: int) -> int:
    """Return the volume of a brick, given its dimensions.

    Preconditions: the dimensions are positive and in millimetres
    Postconditions: the output is in cubic millimetres
    """
    return length * width * height


l = 2
w = 3
h = 4
%timeit -r 5 brick_volume(l, w, h)

34.5 ns ± 0.145 ns per loop (mean ± std. dev. of 5 runs, 10,000,000 loops each)


I didn't set the number of loops, so the interpreter
automatically sets it according to the run-time.
The longer the code takes to run,
the fewer loops are necessary to get a precise measurement. In this example,
the interpreter on my machine 'only' made 10 million function calls
in each run, whereas it had previously made 100 million additions per run.

### 2.8.1 Checking growth rates

To check the complexity of an operation,
we measure the run-times for a series of inputs and look at the trend.
That's why the actual run-time values don't matter,
as long as we measure them consistently.
What matters is how the run-times increase, as the inputs get larger.
Often, we keep doubling the input size or value,
as that's a good way to check if an operation has constant or linear complexity.
Here's a little experiment for the addition operation.

In [7]:
left = 2
right = 7
%timeit -r 5 -n 1000000 left + right  # 1 digit
left = 22
right = 77
%timeit -r 5 -n 1000000 left + right  # 2 digits
left = 2222
right = 7777
%timeit -r 5 -n 10000 left + right  # 4 digits
left = 22222222
right = 77777777
%timeit -r 5 -n 10000 left + right  # 8 digits
left = 2222222222222222
right = 7777777777777777
%timeit -r 5 -n 10000 left + right  # 16 digits
left = 22222222222222222222222222222222
right = 77777777777777777777777777777777
%timeit -r 5 -n 10000 left + right  # 32 digits
left = 2222222222222222222222222222222222222222222222222222222222222222
right = 7777777777777777777777777777777777777777777777777777777777777777
%timeit -r 5 -n 10000 left + right  # 64 digits

12.1 ns ± 0.0435 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
12.2 ns ± 0.0437 ns per loop (mean ± std. dev. of 5 runs, 1,000,000 loops each)
20.6 ns ± 0.0133 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
20.6 ns ± 0.0186 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
23.2 ns ± 0.0249 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
24.2 ns ± 0.0433 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
27.7 ns ± 2.58 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)


Run-times remain similar but increase (without doubling) as the numbers get larger and go beyond 64-bits,
which is consistent with most of the computation being done in hardware. As mentioned before,
since we only use 64-bit integers in M269, the run-time will always be within
a certain bound and hence we can treat addition as a constant-time operation.

You may occasionally observe the mean run-times *decrease* as the inputs increase.
A likely reason is that some other processes were running and so the
previous measurements took longer or varied more across runs
(look at the reported standard deviation) than they normally would.

Let's now look at exponentiation.
We want to check if it's linear in the value of the exponent,
so we must double the value, not the number of digits.
Doubling the exponent quickly leads to multiplying integers that don't fit in 64&nbsp;bits,
but I'll do it anyway to see how the run-times increase.

In [8]:
base = 5
%timeit -r 5 -n 10000 base ** 1
%timeit -r 5 -n 10000 base ** 2
%timeit -r 5 -n 10000 base ** 4
%timeit -r 5 -n 10000 base ** 8
%timeit -r 5 -n 10000 base ** 16
%timeit -r 5 -n 10000 base ** 32
%timeit -r 5 -n 10000 base ** 64

17.6 ns ± 0.0339 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
17.4 ns ± 0.0421 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
28.8 ns ± 0.0345 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
39.3 ns ± 0.193 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
52.2 ns ± 1.43 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
75.7 ns ± 0.178 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)
111 ns ± 2.5 ns per loop (mean ± std. dev. of 5 runs, 10,000 loops each)


Overall, the run-time keeps increasing,
but it's not doubling whenever the exponent doubles.
The interpreter or the hardware is using a more efficient algorithm
than repeatedly multiplying the base with itself.
(You will see such an algorithm in Chapter&nbsp;13.)

<div class="alert alert-warning">
<strong>Note:</strong> To see the growth of run-times, keep doubling the input values or input sizes.
</div>

The fact that exponentiation has a lower complexity than linear in my
computational environment (and yours too, probably) doesn't change our
assumption. When analysing algorithms involving exponentiation,
we shall still assume it's linear.
The whole point of making these assumptions is to guarantee that you and your
700+ fellow M269&nbsp;students obtain the same complexity for the same algorithm,
and not different ones, depending on everyone's computational environment.
You can imagine the confusion this would cause in forums, tutorials and TMAs.

A final note: in general, measuring the run-time on small inputs isn't very useful.
First, the computational environment may make some optimisations for small
values, which means the timings won't fit a clear growth rate.
Second, even without optimisations the run-times may be so short that
they will all be very similar, and look like constant run-time, or may fluctuate due to measuring errors.
Third, we're interested in how algorithms cope with large, not small, inputs.

<div class="alert alert-warning">
<strong>Note:</strong> In general, don't measure run-times for very small inputs.
</div>

#### Exercise 2.8.1

When measuring the run-times of exponentiation,
would  `base = 1` be a good choice? Explain why or why not.

_Write your answer here._

[Hint](../31_Hints/Hints_02_8_01.ipynb)
[Answer](../32_Answers/Answers_02_8_01.ipynb)

#### Exercise 2.8.2

To practice measuring run-times and editing notebooks, choose an operation
(other than addition and exponentiation) from Section&nbsp;2.2
and check whether its run-time is linear as the value or size of the operands increases.
Add a code cell below this paragraph for your experiment.

Use the same number of runs and loops for each measurement, but different from
what I used for addition. Since arithmetic operations take nanoseconds,
choose at least one million loops to make the measurement relatively accurate.
Since Python may optimise operations for values that fit in one byte,
start the experiment with values grater than 255.

[Hint](../31_Hints/Hints_02_8_02.ipynb)
[Answer](../32_Answers/Answers_02_8_02.ipynb)

⟵ [Previous section](02_7_complexity.ipynb) | [Up](02-introduction.ipynb) | [Next section](02_9_summary.ipynb) ⟶